In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders.wikipedia import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

**Loader**

In [3]:
loader = WikipediaLoader(
    "Anthony_Hopkins",
    load_max_docs=1,
    doc_content_chars_max=40000
)
docs = loader.load()

In [4]:
len(docs)

1

In [5]:
len(docs[0].page_content)

38591

In [6]:
print(docs[0].page_content)

Sir Philip Anthony Hopkins (born 31 December 1937) is a Welsh actor. One of Britain's most recognisable and prolific actors, he is known for his performances on the screen and stage. Hopkins has received numerous accolades, including two Academy Awards, four BAFTA Awards, two Primetime Emmy Awards, and a Laurence Olivier Award. He has also received the Cecil B. DeMille Award in 2005 and the BAFTA Fellowship for lifetime achievement in 2008. He was knighted by Queen Elizabeth II for his services to drama in 1993.
After graduating from the Royal Welsh College of Music & Drama in 1957, Hopkins trained at RADA (the Royal Academy of Dramatic Art) in London. He was then spotted by Laurence Olivier, who invited him to join the Royal National Theatre in 1965. Productions at the National included King Lear (his favourite Shakespeare play), Coriolanus, Macbeth, and Antony and Cleopatra. In 1985, he received acclaim and a Laurence Olivier Award for his performance in the David Hare play Pravda. H

**Splitter**

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, 
    chunk_overlap=50
)
all_splits = text_splitter.split_documents(docs)


In [8]:
print(f"Split Wikipedia page into {len(all_splits)} sub-documents.")

Split Wikipedia page into 182 sub-documents.


In [9]:
all_splits[0]

Document(metadata={'title': 'Anthony Hopkins', 'summary': "Sir Philip Anthony Hopkins (born 31 December 1937) is a Welsh actor. One of Britain's most recognisable and prolific actors, he is known for his performances on the screen and stage. Hopkins has received numerous accolades, including two Academy Awards, four BAFTA Awards, two Primetime Emmy Awards, and a Laurence Olivier Award. He has also received the Cecil B. DeMille Award in 2005 and the BAFTA Fellowship for lifetime achievement in 2008. He was knighted by Queen Elizabeth II for his services to drama in 1993.\nAfter graduating from the Royal Welsh College of Music & Drama in 1957, Hopkins trained at RADA (the Royal Academy of Dramatic Art) in London. He was then spotted by Laurence Olivier, who invited him to join the Royal National Theatre in 1965. Productions at the National included King Lear (his favourite Shakespeare play), Coriolanus, Macbeth, and Antony and Cleopatra. In 1985, he received acclaim and a Laurence Olivie

**Embeddings**

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

**Vector Store**

In [11]:
vector_store = InMemoryVectorStore(embeddings)

In [12]:
document_ids = vector_store.add_documents(documents=all_splits)
document_ids[:10]

['f8656980-ae80-44f9-9f09-ab4eb0115491',
 'ba5772ef-fdec-4571-968d-ad9191aaec17',
 '421980aa-fc53-4bbd-be38-519275ca47d1',
 'e8a2fe84-c247-4255-acc1-24139a19cd15',
 'bc4097c7-a075-4f23-868f-f4f9adb15ad5',
 '3f1ded3e-c75a-46f4-80ef-e3870daf5ee3',
 '53db4025-05bf-4246-a92f-7cadfef3ed66',
 '616c4b20-8942-47a7-ae02-7792eecc7032',
 '5a7eb33c-35f1-407a-9be6-588871bc155c',
 'd5e980e8-5929-4bd7-b03a-a91425b1bbc2']

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

**Prompt**

In [14]:
template = ChatPromptTemplate([
    ("system", "You are an assistant for question-answering tasks."),
    ("human", "Use the following pieces of retrieved context to answer the question. "
              "If you don't know the answer, just say that you don't know. " 
              "Use three sentences maximum and keep the answer concise. "
              "\n# Question: \n-> {question} "
              "\n# Context: \n-> {context} "
              "\n# Answer: "),
])

In [15]:
template.invoke(
    {"context": "##CONTEXT##", "question": "##QUESTION##"}
).to_messages()

[SystemMessage(content='You are an assistant for question-answering tasks.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. \n# Question: \n-> ##QUESTION## \n# Context: \n-> ##CONTEXT## \n# Answer: ", additional_kwargs={}, response_metadata={})]

In [16]:
def format_docs(docs):
    formatted = "\n\n-> ".join(doc.page_content for doc in docs)
    return formatted

**Generation**

In [17]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [18]:
question = "When The Silence of the Lambs was released?"
context = format_docs(retriever.invoke(question))
messages = template.invoke({'question' : question, 'context' : context}).to_messages()
answer = llm.invoke(messages)

In [19]:
print(messages[1].content)

Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. 
# Question: 
-> When The Silence of the Lambs was released? 
# Context: 
-> Hopkins won acclaim among critics and audiences as the cannibalistic serial killer Hannibal Lecter in The Silence of the Lambs, for which he won the Academy Award for Best Actor in 1991, with Jodie Foster as Clarice Starling, who also won for Best Actress. The film won Best Picture, Best Director

-> Actress. The film won Best Picture, Best Director and Best Adapted Screenplay, and Hopkins also picked up his first BAFTA for Best Actor. Hopkins reprised his role as Lecter twice; in Ridley Scott's Hannibal (2001), and Red Dragon (2002). His original portrayal of the character in The Silence of

-> portrayal of the character in The Silence of the Lambs has been labelled by the AFI as the number-one film villain. Director Jonathan

In [20]:
answer

AIMessage(content='The Silence of the Lambs was released in 1991.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 288, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c4f1460-9632-4371-b7d3-61bef6b992f9-0', usage_metadata={'input_tokens': 288, 'output_tokens': 14, 'total_tokens': 302, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

**LCEL**

In [21]:
rag_chain = ( 
    RunnableParallel(
        context = retriever | format_docs, 
        question = RunnablePassthrough() 
    )
    | template 
    | llm 
)

In [22]:
rag_chain.invoke("When he was born?")

AIMessage(content='Philip Anthony Hopkins was born on 31 December 1937.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 235, 'total_tokens': 249, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d9c8393-8ea7-4a35-8d52-5bef8b4fa52c-0', usage_metadata={'input_tokens': 235, 'output_tokens': 14, 'total_tokens': 249, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})